# Loading data

In [1]:
import pandas as pd 
import numpy as np

#agar mudah, letakkan file data dalam satu folder dengan file jupiter notebook nya
#jenis file csv utf-8

filedata = 'scrapingBisnisSustain'
#filedata = 'scrapingBisnisTrend'

dataSB = pd.read_csv(filedata+".csv")

dataSB.head()

,link,textdata
0,https://www.msn.com/en-ph/news/opinion/how-to-...,Editor’s note: The opinions in this article ...
1,https://www.marketwatch.com/story/how-to-susta...,The U.S. economy took a remarkable bounce th...
2,https://www.businessinsider.com/3-critical-way...,\nSubscriber\nAccount active\n s...
3,https://www.businessinsider.com/nissans-uk-bus...,\nSubscriber\nAccount active\n s...
4,https://www.businessinsider.com/younger-adults...,\r\nSubscriber\r\nAccount active\r\n ...


# lower case

In [2]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas
dataSB['textdata'] = dataSB['textdata'].str.lower()

print('Case Folding Result : \n')
print(dataSB['textdata'].head(5))

Case Folding Result : 

0      editor’s note: the opinions in this article ...
1      the u.s. economy took a remarkable bounce th...
2     \nsubscriber\naccount active\n              s...
3     \nsubscriber\naccount active\n              s...
4     \r\nsubscriber\r\naccount active\r\n         ...
Name: textdata, dtype: object


# Tokenizing
Memecah teks menjadi perkata, dan membersihkan simbol-simbol

In [3]:
import string 
import re #regex library

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

# ------ Tokenizing ---------

def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
dataSB['textdata'] = dataSB['textdata'].apply(remove_tweet_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

dataSB['textdata'] = dataSB['textdata'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

dataSB['textdata'] = dataSB['textdata'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

dataSB['textdata'] = dataSB['textdata'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

dataSB['textdata'] = dataSB['textdata'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

dataSB['textdata'] = dataSB['textdata'].apply(remove_singl_char)

# NLTK word rokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

dataSB['textdata_tokens'] = dataSB['textdata'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n') 
print(dataSB['textdata_tokens'].head())

Tokenizing Result : 

0    [editors, note, the, opinions, in, this, artic...
1    [the, us, economy, took, remarkable, bounce, t...
2    [subscriber, account, active, since, free, sub...
3    [subscriber, account, active, since, free, sub...
4    [subscriber, account, active, since, free, sub...
Name: textdata_tokens, dtype: object


# NLTK calc frequency distribution
Menghitung frequency tiap kata

In [4]:
# NLTK calc frequency distribution
def freqDist_wrapper(text):
    return FreqDist(text)

dataSB['textdata_tokens_fdist'] = dataSB['textdata_tokens'].apply(freqDist_wrapper)

print('Frequency Tokens : \n') 
print(dataSB['textdata_tokens_fdist'].head().apply(lambda x : x.most_common()))

Frequency Tokens : 

0    [(the, 71), (and, 44), (of, 44), (to, 41), (in...
1    [(the, 44), (and, 28), (to, 20), (in, 18), (of...
2    [(to, 30), (you, 24), (and, 23), (the, 23), (y...
3    [(it, 12), (the, 11), (that, 9), (in, 9), (not...
4    [(the, 14), (to, 9), (and, 8), (care, 8), (of,...
Name: textdata_tokens_fdist, dtype: object


# Stopwords
Membersihkan teks dari kata-kata yang termasuk ke dalam stopwords

In [5]:
from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('english')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(['from', 'subject', 're', 'edu', 'use', 'sustain', 'sustainability','sustainable', 'work', 'mr',
                       'singapore', 'us', 'china',
                      'business', 'trend'])

# convert list to dictionary
list_stopwords = set(list_stopwords)

#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

dataSB['textdata_tokens_WSW'] = dataSB['textdata_tokens'].apply(stopwords_removal) 

print(dataSB['textdata_tokens_WSW'].head())

0    [editors, note, opinions, article, authors, pu...
1    [economy, took, remarkable, bounce, past, mont...
2    [subscriber, account, active, since, free, sub...
3    [subscriber, account, active, since, free, sub...
4    [subscriber, account, active, since, free, sub...
Name: textdata_tokens_WSW, dtype: object


# Normalisasi
Mengganti kata-kata tertentu dengan kata lain yang lebih tepat

In [6]:
normalizad_word = pd.read_excel("normalisasi.xlsx")

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

dataSB['textdata_normalized'] = dataSB['textdata_tokens_WSW'].apply(normalized_term)

dataSB['textdata_normalized'].head(10)

0    [editors, note, opinions, article, authors, pu...
1    [economy, took, remarkable, bounce, past, mont...
2    [subscriber, account, active, since, free, sub...
3    [subscriber, account, active, since, free, sub...
4    [subscriber, account, active, since, free, sub...
5    [book, based, ideas, associated, labor, group,...
6    [ben, casselman, brandon, williams, supposed, ...
7    [keith, bradsher, elaine, yu, russell, goldman...
8    [nelson, schwartz, gillian, friedman, american...
9    [earlier, year, proprietary, data, led, make, ...
Name: textdata_normalized, dtype: object

# Stemmer NLTK

In [7]:
from nltk.stem import PorterStemmer
stemming = PorterStemmer()

my_list = ['customer', 'generously', 'frightens']

# Using a Python list comprehension method to apply to all words in my_list

print ([stemming.stem(word) for word in my_list])

['custom', 'gener', 'frighten']


In [8]:
from nltk.stem.snowball import SnowballStemmer
stemming = SnowballStemmer("english")


my_list = ['customers', 'generously', 'digital']

# Using a Python list comprehension method to apply to all words in my_list

print ([stemming.stem(word) for word in my_list])

['custom', 'generous', 'digit']


In [9]:
import pandas as pd

#from nltk.stem import PorterStemmer
#stemming = PorterStemmer()

from nltk.stem.snowball import SnowballStemmer
stemming = SnowballStemmer("english")

def stem_list(row):
    my_list = row['textdata_normalized']
    stemmed_list = [stemming.stem(word) for word in my_list]
    return (stemmed_list)

dataSB['textdata_tokens_stemmed'] = dataSB.apply(stem_list, axis=1)
print(dataSB['textdata_tokens_stemmed'])

0      [editor, note, opinion, articl, author, publis...
1      [economi, took, remark, bounc, past, month, ac...
2      [subscrib, account, activ, sinc, free, subscri...
3      [subscrib, account, activ, sinc, free, subscri...
4      [subscrib, account, activ, sinc, free, subscri...
                             ...                        
995    [youv, work, hard, life, your, brink, retir, t...
996    [young, peopl, love, plant, nurseri, owner, wa...
997    [cultur, might, posit, brand, messag, might, p...
998    [youv, work, hard, put, countless, hour, make,...
999    [zhangy, juli, employe, harvest, tomato, intel...
Name: textdata_tokens_stemmed, Length: 1000, dtype: object


In [10]:
#batas coba steamer NLTK

In [11]:
dataSB['textdata_tokens_stemmed'][0:5]

0    [editor, note, opinion, articl, author, publis...
1    [economi, took, remark, bounc, past, month, ac...
2    [subscrib, account, activ, sinc, free, subscri...
3    [subscrib, account, activ, sinc, free, subscri...
4    [subscrib, account, activ, sinc, free, subscri...
Name: textdata_tokens_stemmed, dtype: object

In [12]:
term_dict = {}

for document in dataSB['textdata_normalized']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))

33992


In [13]:
for i in range(len(dataSB)):
        a=dataSB.iloc[i][5]
        document.append(a)
        
document[0:5]

['zhangye', 'july', 'employees', 'harvest', 'tomatoes']

# normalized again

In [14]:
normalizad_word = pd.read_excel("normalisasi.xlsx")

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

dataSB['textdata_tokens_stemmed'] = dataSB['textdata_tokens_stemmed'].apply(normalized_term)

dataSB['textdata_tokens_stemmed'].head(10)

0    [editor, note, opinion, articl, author, publis...
1    [economi, took, remark, bounc, past, month, ac...
2    [subscrib, account, activ, sinc, free, subscri...
3    [subscrib, account, activ, sinc, free, subscri...
4    [subscrib, account, activ, sinc, free, subscri...
5    [book, base, idea, associ, labor, group, earli...
6    [ben, casselman, brandon, william, suppos, spe...
7    [keith, bradsher, elain, yu, russel, goldman, ...
8    [nelson, schwartz, gillian, friedman, american...
9    [earlier, year, proprietari, data, led, make, ...
Name: textdata_tokens_stemmed, dtype: object

# stopwords again

In [15]:
from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('english')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(['from', 'subject', 're', 'edu', 'use', 'business', 'trend', 'company', 'new', 'said', 'percent',
                       'economy', 'econom', 'economi', 'trump', 'per', 'cent',
                       'would', 'could', 'also', 'dont',
                       'mani', 'cnbc', 'get', 'make', 'million',
                       'help',
                       'like', 'year', 'mr', 'work'])

# convert list to dictionary
list_stopwords = set(list_stopwords)

#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

dataSB['textdata_tokens_stemmed'] = dataSB['textdata_tokens_stemmed'].apply(stopwords_removal) 

print(dataSB['textdata_tokens_stemmed'].head())

0    [editor, note, opinion, articl, author, publis...
1    [took, remark, bounc, past, month, accomplish,...
2    [subscrib, account, activ, sinc, free, subscri...
3    [subscrib, account, activ, sinc, free, subscri...
4    [subscrib, account, activ, sinc, free, subscri...
Name: textdata_tokens_stemmed, dtype: object


In [16]:
doc_clean = dataSB['textdata_tokens_stemmed']
doc_clean

0      [editor, note, opinion, articl, author, publis...
1      [took, remark, bounc, past, month, accomplish,...
2      [subscrib, account, activ, sinc, free, subscri...
3      [subscrib, account, activ, sinc, free, subscri...
4      [subscrib, account, activ, sinc, free, subscri...
                             ...                        
995    [youv, hard, life, brink, retir, troubl, thing...
996    [young, people, love, plant, nurseri, owner, w...
997    [cultur, might, posit, brand, messag, might, p...
998    [youv, hard, put, countless, hour, today, defi...
999    [zhangy, juli, employe, harvest, tomato, intel...
Name: textdata_tokens_stemmed, Length: 1000, dtype: object

In [17]:
import gensim
from gensim import corpora

dictionary = corpora.Dictionary(doc_clean)
print(dictionary)

doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

Dictionary(23141 unique tokens: ['aboitiz', 'accept', 'accustom', 'action', 'activ']...)


# LDA model using gensim
Proses topic modeling dengan LDA gensim

In [18]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

total_topics = 5 # jumlah topik yang akan di extract
number_words = 12 # jumlah kata per topik

# main topic extracted

In [19]:
# Running and Trainign LDA model on the document term matrix.
# mungkin agak lama
lda_model = Lda(doc_term_matrix, num_topics=total_topics, id2word = dictionary, passes=50)

lda_model.show_topics(num_topics=total_topics, num_words=number_words)

[(0,
  '0.007*"people" + 0.007*"time" + 0.006*"need" + 0.006*"one" + 0.006*"employe" + 0.006*"organisation" + 0.006*"leader" + 0.005*"manag" + 0.005*"team" + 0.005*"way" + 0.004*"chang" + 0.004*"many"'),
 (1,
  '0.011*"rate" + 0.011*"fed" + 0.009*"cnn" + 0.007*"market" + 0.006*"cut" + 0.005*"reserv" + 0.005*"jone" + 0.005*"bank" + 0.005*"dow" + 0.005*"updat" + 0.005*"time" + 0.005*"indic"'),
 (2,
  '0.010*"customer" + 0.007*"innovation" + 0.007*"data" + 0.006*"strategi" + 0.006*"market" + 0.006*"product" + 0.005*"valu" + 0.005*"technology" + 0.004*"invest" + 0.004*"develop" + 0.004*"competit" + 0.004*"one"'),
 (3,
  '0.015*"market" + 0.008*"growth" + 0.007*"industri" + 0.006*"report" + 0.006*"global" + 0.005*"product" + 0.005*"increas" + 0.004*"unit" + 0.004*"invest" + 0.004*"state" + 0.004*"bank" + 0.004*"stock"'),
 (4,
  '0.005*"govern" + 0.005*"small" + 0.004*"worker" + 0.004*"pandem" + 0.004*"month" + 0.004*"people" + 0.004*"time" + 0.004*"need" + 0.004*"store" + 0.004*"loan" + 0.0

In [20]:
# Word Count of Topic Keywords

from collections import Counter
topics = lda_model.show_topics(formatted=False)
data_flat = [w for w_list in doc_clean for w in w_list]
counter = Counter(data_flat)

out = []
for i, topic in topics:
    for word, weight in topic:
        out.append([word, i , weight, counter[word]])

df_imp_wcount = pd.DataFrame(out, columns=['word', 'topic_id', 'importance', 'word_count']) 
print(df_imp_wcount)

            word  topic_id  importance  word_count
0         people         0    0.007332        2181
1           time         0    0.006923        2483
2           need         0    0.006284        2149
3            one         0    0.006146        2148
4        employe         0    0.006045        1600
5   organisation         0    0.005632        1488
6         leader         0    0.005584        1306
7          manag         0    0.005084        1627
8           team         0    0.004994        1169
9            way         0    0.004747        1400
10          rate         1    0.011320         930
11           fed         1    0.011117         486
12           cnn         1    0.008935         385
13        market         1    0.006869        2686
14           cut         1    0.005620         513
15        reserv         1    0.005417         509
16          jone         1    0.005193         239
17          bank         1    0.005173         874
18           dow         1    0

In [21]:
#Dominant topic and its percentage contribution in each topic
def format_topics_sentences(ldamodel=None, corpus=doc_term_matrix, texts=document):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [22]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=doc_term_matrix, texts=doc_clean)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
print(df_dominant_topic.head(10))

   Document_No  Dominant_Topic  Topic_Perc_Contrib  \
0            0             0.0              0.4354   
1            1             4.0              0.5447   
2            2             0.0              0.4876   
3            3             3.0              0.8754   
4            4             0.0              0.3819   
5            5             2.0              0.4795   
6            6             4.0              0.9953   
7            7             4.0              0.6014   
8            8             4.0              0.7630   
9            9             1.0              0.6323   

                                            Keywords  \
0  people, time, need, one, employe, organisation...   
1  govern, small, worker, pandem, month, people, ...   
2  people, time, need, one, employe, organisation...   
3  market, growth, industri, report, global, prod...   
4  people, time, need, one, employe, organisation...   
5  customer, innovation, data, strategi, market, ...   
6  govern, sm

# visualization package pyLDAvis

In [23]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()

In [24]:
import os
LDAvis_data_filepath = os.path.join('ldavis_'+filedata+'_'+str(total_topics))

In [25]:
corpus = [dictionary.doc2bow(text) for text in doc_clean]

In [26]:
# proses ini mungkin agak lama
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

In [27]:
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

In [28]:
# proses ini mungkin agak lama
pyLDAvis.save_html(LDAvis_prepared, 'ldavis_'+filedata+'_'+str(total_topics) +'.html')

In [29]:
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.105181  0.071266       1        1  40.429939
3     -0.012790 -0.123941       2        1  18.176985
4     -0.030086  0.053922       3        1  17.659782
2      0.098340 -0.029574       4        1  16.145185
1     -0.160646  0.028326       5        1   7.588113, topic_info=          Term         Freq        Total Category  logprob  loglift
203     market  2527.000000  2527.000000  Default  30.0000  30.0000
1500      rate   857.000000   857.000000  Default  29.0000  29.0000
1399       fed   436.000000   436.000000  Default  28.0000  28.0000
2907       cnn   345.000000   345.000000  Default  27.0000  27.0000
86    customer  1773.000000  1773.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
356       time   177.701187  2396.831787   Topic5  -5.3802  -0.0232
776      right   128.358795   911.536255   Topic5  -5.7055   0.6183
504     growth   140.169662  1457.646484   Topic5  -5.6175   0.2369
2234    expect   117.331055   667.973145   Topic5  -5.7953   0.8393
1393      data   118.444054  1345.182617   Topic5  -5.7859   0.1487

[437 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
997       1  0.252430      across
997       2  0.427189      across
997       3  0.128863      across
997       4  0.167698      across
997       5  0.021183      across
...     ...       ...         ...
2905      3  0.027218        zoom
2905      4  0.753037        zoom
4194      3  0.047005  zuckerberg
4194      4  0.141015  zuckerberg
4194      5  0.799087  zuckerberg

[1126 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 5, 3, 2])

# References

https://yunusmuhammad007.medium.com/text-preprocessing-menggunakan-pandas-nltk-dan-sastrawi-untuk-large-dataset-5fb3c0a88571

https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0

